## Set up the Colab instance to run on a GPU accelerator


Navigate to Edit→Notebook Settings and select "GPU" from the "Hardware accelerator" drop-down menu.

## Install dependencies, download the model, set up your PYTHONPATH

From here on, you'll start seeing a mix of code and Linux system commands. System commands are prefixed by a shebang `!`, which tells this notebook to execute them on the command line.

### Install required Python packages

This may take 2-3 minutes.

In [ ]:
pip install humanfriendly jsonpickle ultralytics

### Download the MegaDetector model files

We'll download both MegaDetector v5a and v5b.  See the [release notes](https://github.com/agentmorris/MegaDetector/releases/tag/v5.0) for information about the differences between the two models.

We have to use the not-quite-official releases of each, because the version of PyTorch supported by MD is no longer available via pip, and conda is... difficult in Colab.

In [ ]:
!wget -O /content/md_v5a.0.0.pt https://lila.science/public/md_rebuild/md_v5a.0.0_rebuild_pt-1.12_zerolr.pt
!wget -O /content/md_v5b.0.0.pt https://lila.science/public/md_rebuild/md_v5b.0.0_rebuild_pt-1.12_zerolr.pt

--2023-09-30 16:14:23--  https://lila.science/public/md_rebuild/md_v5a.0.0_rebuild_pt-1.12_zerolr.pt
Resolving lila.science (lila.science)... 20.83.252.133
Connecting to lila.science (lila.science)|20.83.252.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281461925 (268M)
Saving to: ‘/content/md_v5a.0.0.pt’

/content/md_v5a.0.0 100%[===================>] 268.42M  86.0MB/s    in 3.4s    

2023-09-30 16:14:26 (79.0 MB/s) - ‘/content/md_v5a.0.0.pt’ saved [281461925/281461925]

--2023-09-30 16:14:26--  https://lila.science/public/md_rebuild/md_v5b.0.0_rebuild_pt-1.12_zerolr.pt
Resolving lila.science (lila.science)... 20.83.252.133
Connecting to lila.science (lila.science)|20.83.252.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281461925 (268M)
Saving to: ‘/content/md_v5b.0.0.pt’

/content/md_v5b.0.0 100%[===================>] 268.42M  79.3MB/s    in 3.7s    

2023-09-30 16:14:30 (73.1 MB/s) - ‘/content/md_v5b.0.0.pt’ saved [2814

### Clone the required git repos
This will copy the latest version of the MegaDetector and YOLOv5 repos, which are required to run MegaDetector.

In [ ]:
!rm -rf /content/MegaDetector
!rm -rf /content/yolov5
!git clone https://github.com/agentmorris/MegaDetector /content/MegaDetector
!git clone https://github.com/ultralytics/yolov5 /content/yolov5

Cloning into '/content/MegaDetector'...
remote: Enumerating objects: 17246, done.
remote: Counting objects: 100% (2092/2092), done.
remote: Compressing objects: 100% (719/719), done.
remote: Total 17246 (delta 1381), reused 2079 (delta 1370), pack-reused 15154
Receiving objects: 100% (17246/17246), 187.14 MiB | 33.15 MiB/s, done.
Resolving deltas: 100% (10577/10577), done.
Cloning into '/content/yolov5'...
remote: Enumerating objects: 16000, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 16000 (delta 20), reused 20 (delta 12), pack-reused 15967
Receiving objects: 100% (16000/16000), 14.59 MiB | 18.82 MiB/s, done.
Resolving deltas: 100% (10986/10986), done.


### Set `PYTHONPATH` to include `MegaDetector` and `yolov5`

Add cloned git folders to the `PYTHONPATH` environment variable so that we can import their modules from any working directory.


In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/MegaDetector"
os.environ['PYTHONPATH'] += ":/content/yolov5"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/train_dataset_altai.zip

Archive:  /content/drive/MyDrive/train_dataset_altai.zip
replace broken_imgs/PICT0082.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## MegaDetector batch processing

This step executes the Python script `run_detector_batch.py` from the MegaDetector repo. It has three mandatory arguments and one optional:

1. Path to the MegaDetector model file
2. A folder containing images.  This notebook points to the folder where we just put our Snapshot Serengeti images; if your images were already on Google Drive, replace `[Image_Folder]` with your folder name.
3. The output JSON file location and name.

In [ ]:
images_dir = '/content/Фотоловушка Иониха'

# Choose a location for the output JSON file
output_file_path = '/content/drive/My Drive/fotolovuska/results.json'

# Run the detection script

There are actually two variants of MegaDetector v5, called "v5a" and "v5b".  By default this notebook runs MDv5a; change "md_v5a.0.0.pt" to "md_v5b.0.0.pt" to run MDv5b instead.

Both run at the same speed; if you are in a Colab session with a GPU accelerator, you should be able to process around four images per second.

In [ ]:
!python /content/MegaDetector/detection/run_detector_batch.py md_v5a.0.0.pt "$images_dir" "$output_file_path" --recursive --output_relative_filenames --quiet

420 image files found in the input directory
PyTorch reports 1 available CUDA devices
GPU available: True
Imported YOLOv5 from PYTHONPATH
Using PyTorch version 2.0.1+cu118
Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients, 207.9 GFLOPs
Sending model to GPU
Loaded model in 16.95 seconds
Loaded model in 16.95 seconds
100% 420/420 [01:40<00:00,  4.19it/s]
Finished inference for 420 images in 2 minutes and 1.38 seconds (3.46 images per second)
Output file saved at /content/drive/My Drive/fotolovuska/results.json
Done!


In [ ]:
# Render bounding boxes on our images
visualization_dir = '/content/visualized_images'
!python /content/MegaDetector/md_visualization/visualize_detector_output.py "$output_file_path" "$visualization_dir" --confidence 0.2 --images_dir "$images_dir"

In [ ]:
# Show the images with bounding boxes in Colab
import os
from PIL import Image

for viz_file_name in os.listdir(visualization_dir):
  print(viz_file_name)
  im = Image.open(os.path.join(visualization_dir, viz_file_name))
  display(im)

NameError: ignored

In [ ]:
import json
import numpy as np
import pandas as pd
import argparse
from PIL import Image as ImagePIL
import os
import cv2
import shutil
import json
import csv
# Открытие файла для чтения

#from google.colab import drive
#drive.mount('/content/drive')

path_in= output_file_path
path_out='/content/drive/My Drive/fotolovuska/result.csv'

In [ ]:
THRESHOLD_BLUR = 200
THRESHOLD_IDENTICAL_BYTES = 65000
RESIZE_SCALE_FACTOR = 0.1

BROKEN_DIR = "broken"
EMPTY_DIR = "empty"
ANIMAL_DIR = "animal"

def check_file_size(filename):
    filesize = os.stat(filename).st_size
    if os.stat(filename).st_size:
        raise SyntaxError("Zero size file")
    return filesize

def test_PIL(filename):
    img = ImagePIL.open(filename)
    img.verify()  # используем для проверки встроенную ф-ю verify()
    img.close()

    img = ImagePIL.open(filename)
    img.transpose(ImagePIL.FLIP_LEFT_RIGHT) # делаем преобразование изображение. Выдает ошибку если есть некоторые дефекты в файле
    img.close()

def calc_number_identical_bytes(filename):
    f = open(filename, "rb")
    bin_data = f.read()
    f.close()
    n = 1
    maxnum = 0
    prev = None

    for i in bin_data:
        if prev == i:
            n += 1
        else:
            if n > maxnum:
                maxnum = n

            n = 1
            prev = i
    if n > maxnum:
        maxnum = n

    return maxnum

def bad_color_areas(gray_image): #функция искать одноцветные области брака
    b=0
    n=np.array(gray_image)
    unique, counts = np.unique(np.ravel(gray_image), return_counts=True)

    for i in range(len(n[:,0])):
        if len(np.unique(np.array(n[i,:])))==1 :
            uno=np.unique(np.array(n[i,:]), return_counts=False)
            if uno!=0:
                if uno!=255:
                    b+=1

    #print(b)
    br=0
    if b>25:
        br=1

    elif len(np.array(unique))<7:
        br=1
    elif (max(np.array(counts))/len(np.ravel(gray_image)))>0.25:

        if np.mean(n)<240:
            br=0
            if np.mean(n)<70:
                br=0
            else: br=1

        else: br=1
        # print(max(np.array(counts))/len(n))
        #  br=1

    return br

def koeff(gray_image):#соотношения белого, черного, серединки
    n=np.ravel(gray_image)
    av=np.mean(n)
    koef_bl=sum(n>240)/len(n)
    koef_wt=sum(n<10)/len(n)
    koef_av=(sum(n<(av+10))-sum(n<(av-10)))/len(n)
    return (av,koef_bl,koef_wt,koef_av)

def color_channel_bad(img): #ошибки в цветовых каналах
    r = img[:,:,0]
    g= img[:,:,1]
    b= img[:,:,2]
    rr=np.ravel(r)
    gr=np.ravel(g)
    br=np.ravel(b)
    #print(np.mean(rr),np.mean(br),np.mean(gr))
    if np.mean(rr)>2*np.mean(gr) or np.mean(rr)>2*sum(br):
        br=1
    elif np.mean(gr)>2*np.mean(rr) or np.mean(gr)>2*np.mean(br):
        br=1
    elif np.mean(br)>2*np.mean(gr) or np.mean(br)>2*np.mean(rr):
        br=1
    else:
        br=0
    return br


def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def copy_file(full_filename, target_dir, sub_dir):
    full_dir = os.path.join(target_dir, sub_dir)
    os.makedirs(full_dir, exist_ok=True)
    shutil.copy2(full_filename, full_dir)


def processing_dataset(srcpath, targetpath, jsonpath):

    # TODO добавить проверку на существование уже таких целевых папок
    full_broken_dir = os.path.join(targetpath, BROKEN_DIR)
    full_empty_dir = os.path.join(targetpath, EMPTY_DIR)
    full_animal_dir = os.path.join(targetpath, ANIMAL_DIR)

    # Создаем директории для каждого класса
    os.makedirs(full_broken_dir, exist_ok=True)
    os.makedirs(full_empty_dir, exist_ok=True)
    os.makedirs(full_animal_dir, exist_ok=True)


    with open(jsonpath, 'r') as file:
        # Parse JSON data
        jsondata = json.load(file)


    csv_data = []
    num_file = len(jsondata['images'])
    current_count = 0
    break_count = -1 # для отладки установить сколько строк обработать в json файле или -1 для всех

    for json_image in jsondata['images'] :
        if not break_count:
            break
        break_count -= 1
        current_count += 1
        #path = root.split(os.sep)
        img_filename_json = json_image['file']
        # отделяем относительный путь от имени файла изображения
        path, file = os.path.split(img_filename_json)

        full_filename = os.path.join(srcpath, img_filename_json)

        print(full_filename)

        num_animals=0
        detections = json_image['detections']
        for i in range(len(detections)):
            kategoria=(detections[i]['category'])
            uverennost= (detections[i]['conf'])
            if int(kategoria) == 1 and float(uverennost)>0.7:
                num_animals += 1
                print("{}: {:.2f}".format("Обнаружены животные с уверенностью", uverennost))
            #i+=1

        if num_animals > 0:
            copy_file(full_filename, full_animal_dir, path) # копируем файл в директорию животных
            csv_data.append([img_filename_json,0,0,1])
            continue



        try:
            # Техническая проверка возможности считывания файла и работы с ним библиотекой PIL
            test_PIL(full_filename)
        except:
            print("test_PIL: обнаружена ошибка в файле!")
            copy_file(full_filename, full_broken_dir, path) # копируем файл в директорию сломанных
            csv_data.append([img_filename_json,1,0,0])
        else: # если файл нормально считывается

            image = cv2.imdecode(np.fromfile(full_filename, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
            #image = cv2.imread(full_filename)
            # TODO знась ОЧЕНЬ НУЖНО выловить сообщения об ошибке вида:
            # Corrupt JPEG data:
            # .imread() посылает их в поток сообщений, но не вызывает никаких ошибок и исключений!
            # это решит проблему с частично битыми изображениями, где остальные методы не сработали

            # Наиболее частая проблема в датасете - это размытые и засвеченные кадры
            # Проанализируем размытость изображения, для этого сперва преобразуем изображение в градации серого


            if len(image.shape) == 2: # проверка если изображение с одним каналом
                gray_image = image
            else: # если не делать проверку, то ч/б изображения вызывают ошибку здесь
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # уменьшаем размер
            image = cv2.resize(image,(0, 0), fx=RESIZE_SCALE_FACTOR, fy=RESIZE_SCALE_FACTOR)

            # сворачиваем изображение с помощью следующего ядра, размерностью 3х3
            # 0  1  0
            # 1 -4  1
            # 0  1  0
            # и вычислим дисперсию Лапласа, чем она меньше, тем более размытое изображение
            # Оператор Лапласа выделяет области изображения, содержащие быстрые изменения интенсивности

            fm = variance_of_laplacian(gray_image)

            if fm < THRESHOLD_BLUR:
                text = "Blurry"
                print("{}: {:.2f}".format("Размытое изображение variance_of_laplacian", fm))

                copy_file(full_filename, full_broken_dir, path) # копируем файл в директорию сломанных
                csv_data.append([img_filename_json,1,0,0])
                continue
            else:
                text = "Not Blurry"

            # Проверим, файл на бинарном уровне на наличие длинных повторяющихся последовательностей (битый канал или однородный цвет)

            number_identical_bytes = calc_number_identical_bytes(full_filename)
            if number_identical_bytes > THRESHOLD_IDENTICAL_BYTES: # если количество повторяющихся байт в файле больше заданного порога
                print("{}: {:d}".format("Вероятно поврежденный файл! calc_number_identical_bytes ", number_identical_bytes))
                copy_file(full_filename, full_broken_dir, path)# копируем файл в директорию сломанных
                csv_data.append([img_filename_json,1,0,0])
                continue




            # здесь идеи Кати
            br = 0
            av, koef_bl, koef_wt, koef_av = koeff(gray_image)
            if av <=10 or av>=240:
                br=1
                print("Broken: av <=10 or av>=240")
            elif koef_bl>0.8 or koef_av>0.8 or koef_wt>0.8:
                br=1
                print("Broken: koef_bl>0.8 or koef_av>0.8 or koef_wt>0.8")
            else :
                br = bad_color_areas(gray_image)
                print("bad_color_areas = "+str(br))
                if br == 0: # след проверка
                    br = color_channel_bad(image)
                    print("color_channel_bad = "+str(br))

            if br:
                print("Помещаем в класс Broken.")
                copy_file(full_filename, full_broken_dir, path) # копируем файл в директорию сломанных
                csv_data.append([img_filename_json,1,0,0])
                continue


            print("На изображении не обнаружено животных и нет дефектов. Помещаем в класс Empty.")
            copy_file(full_filename, full_empty_dir, path) # копируем файл в директорию пустых
            csv_data.append([img_filename_json,0,1,0])

            # cv2.putText(image, "{}: {:.2f}".format(text, fm), (30, 30),
            # cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            # cv2.putText(image, "{}: {:.2f}".format("number_identical_bytes", number_identical_bytes), (30, 60),
            # cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            # #cv2.imshow("Image", image) # показать на экране
            # cv2.imwrite(os.path.join(targetpath, file), image)


    with open(os.path.join(targetpath, 'submission.csv'), 'w', encoding='utf-8', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',')
        csv_writer.writerow(['filename','broken','empty','animal'])
        for s in csv_data:
            csv_writer.writerow(s)




    print("\nThe End")
    print("Press any key to continue")
    key = cv2.waitKey(0)

def arg_parser():
    epilog_text = """
    Скрипт фильтрации изображений
    """

    parser = argparse.ArgumentParser(description='Определение дефектных фото', epilog=epilog_text)
    parser.add_argument('srcpath', metavar='SOURCE_PATH', type=str,
                        help='Путь к папке - источнику изображений')
    parser.add_argument('targetpath', metavar='TARGET_PATH', type=str,
                        help='Путь к целевой папке найденных (дефектных) изображений')
    parser.add_argument('json', metavar='JSON_PATH', type=str,
                        help='Путь к json файлу с результатами распознавания животных')
    parser.add_argument("-tb", "--threshold_blur", type=int, default=280,
                        help="Порог 'размытости' изображений (по умолчанию = 280)")
    parser.add_argument("-ni", "--threshold_identical_bytes", type=int, default=65000,
                        help="Порог 'размытости' изображений (по умолчанию = 65000)")

    return parser.parse_args()

def main():
    global ARG
    ARG = arg_parser()

In [ ]:
processing_dataset(images_dir,'/content/итог', output_file_path)

/content/Фотоловушка Иониха/SYER0023.JPG
Размытое изображение variance_of_laplacian: 268.09
/content/Фотоловушка Иониха/SYER0024.JPG
Размытое изображение variance_of_laplacian: 244.74
/content/Фотоловушка Иониха/SYER0025.JPG
Размытое изображение variance_of_laplacian: 243.12
/content/Фотоловушка Иониха/SYER0026.JPG
Обнаружены животные с уверенностью: 0.95
/content/Фотоловушка Иониха/SYER0027.JPG
Обнаружены животные с уверенностью: 0.95
/content/Фотоловушка Иониха/SYER0028.JPG
Обнаружены животные с уверенностью: 0.94
/content/Фотоловушка Иониха/SYER0029.JPG
На изображении не обнаружено животных и нет дефектов. Помещаем в класс Empty.
/content/Фотоловушка Иониха/SYER0030.JPG
На изображении не обнаружено животных и нет дефектов. Помещаем в класс Empty.
/content/Фотоловушка Иониха/SYER0031.JPG
На изображении не обнаружено животных и нет дефектов. Помещаем в класс Empty.
/content/Фотоловушка Иониха/SYER0032.JPG
На изображении не обнаружено животных и нет дефектов. Помещаем в класс Empty.

T